# Astro Vibe Bot — Local Test Notebook

Use this notebook to generate a daily context, preview the channel message for all 12 signs, and optionally send it to a Telegram chat or channel.

Prereqs:
- `.env` must exist with `TELEGRAM_BOT_TOKEN` and `OPENAI_API_KEY`
- Optional: set `RSS_FEED_URL`, `TIMEZONE`, and `TEST_CHAT_ID` in `.env`


In [1]:
import os
import asyncio
from pathlib import Path
from zoneinfo import ZoneInfo

from dotenv import load_dotenv
from openai import OpenAI
from aiogram import Bot

from main import load_signs, generate_daily_context, build_channel_sign_messages

root_dir = Path.cwd()
if not (root_dir / ".env").exists() and (root_dir.parent / ".env").exists():
    root_dir = root_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
RSS_FEED_URL = os.getenv("RSS_FEED_URL")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
TEST_CHAT_ID = os.getenv("TEST_CHAT_ID")
TIMEZONE = os.getenv("TIMEZONE", "UTC")
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")
TELEGRAM_NEWS_CHANNEL = os.getenv("TELEGRAM_NEWS_CHANNEL")
TELEGRAM_NEWS_LIMIT = int(os.getenv("TELEGRAM_NEWS_LIMIT", "20"))
TELETHON_SESSION = os.getenv("TELETHON_SESSION", "telethon.session")
TELETHON_SESSION_STRING = os.getenv("TELETHON_SESSION_STRING")

if not TELEGRAM_BOT_TOKEN or not OPENAI_API_KEY:
    raise RuntimeError("Set TELEGRAM_BOT_TOKEN and OPENAI_API_KEY in .env")

client = OpenAI(api_key=OPENAI_API_KEY)
signs = load_signs()
timezone = ZoneInfo(TIMEZONE)
telegram_source = {
    "api_id": int(TELEGRAM_API_ID) if TELEGRAM_API_ID else None,
    "api_hash": TELEGRAM_API_HASH,
    "channel": TELEGRAM_NEWS_CHANNEL,
    "limit": TELEGRAM_NEWS_LIMIT,
    "session_path": TELETHON_SESSION,
    "session_string": TELETHON_SESSION_STRING,
}


ImportError: cannot import name 'build_all_signs_message' from 'main' (/Users/tetianatorovets/repos/astro-vibe-bot/main.py)

In [2]:
# Generate today's global context
context = await generate_daily_context(
    client,
    signs,
    RSS_FEED_URL,
    OPENAI_MODEL,
    telegram_source=telegram_source,
)
context["global_summary"], list(context["vibes"].keys())[:3]


Signed in successfully as Tetiana; remember to not break the ToS or you will risk an account ban!
[debug] Telegram news context:
- LELÉKA представлять Україну на «Євробаченні-2026» НОВИНА | Підписатися
- ❗️Працівники ТЦК забили до смерті 55-річного чоловіка, — поліція Дніпропетровщини Інформація про смерть чоловіка надійшла сьогодні вночі. Трьох працівників ТЦК затримано. В авто виявили сліди крові. НОВИНА | Підписатися
- Кадри удару по ТЕЦ «Луч» пакетом GMLRS РСЗВ HIMARS сьогодні зранку НОВИНА | Підписатися
- ⚡️ Мусимо пройти найближчі кілька діб, що будуть дуже складними для Києва. У столиці знову прогнозують сильні морози, зокрема, вночі, — Кличко НОВИНА | Підписатися
- В людей шок! У Буковелі почали відключати світло На відео момент, коли курорт переходив на генератори. НОВИНА | Підписатися
- У російському бєлгороді масовий блекаут Губернатор області вже бідкається, що БпЛА атакували місцеву ТЕЦ "Луч" та електропідстанцію "бєлгород". НОВИНА | Підписатися
- ❗️Завтра в усіх регіонах 

('День насичений викликами, але й можливостями для зростання та підтримки один одного.',
 ['Aries', 'Taurus', 'Gemini'])

In [4]:
# Build the daily channel messages (12 separate posts)
channel_messages = build_channel_sign_messages(context, signs)
channel_messages


'Сьогодні ти здатен подолати всі труднощі!\n\nДень насичений викликами, але й можливостями для зростання та підтримки один одного.\n\nОвен: Сьогодні твоє імпульсивне натхнення може відкрити нові горизонти. Не бійтеся діяти сміливо!\n\nТелець: Стабільність і терпіння - твої найсильніші козирі. Тримай курс і не піддавайся паніці!\n\nБлизнюки: Твоя здатність адаптуватися сьогодні стане в пригоді. Залишайся відкритим до нових ідей!\n\nРак: Слухай своє серце і підтримуй близьких. Твоя чутливість створить атмосферу комфорту.\n\nЛев: Сьогодні ти - світло в темряві! Вір у свої сили і веди інших за собою.\n\nДіва: Зосередься на деталях, вони допоможуть тобі знайти рішення. Твоя аналітичність - ключ до успіху.\n\nТерези: Шукай гармонію в спілкуванні з іншими. Твоя дипломатичність допоможе уникнути конфліктів.\n\nСкорпіон: Твоя інтенсивність і стратегічний підхід приведуть до успіху. Дій обережно, але рішуче.\n\nСтрілець: Сьогодні твій оптимізм може надихнути інших. Не бійся ділитися своїми ідеям

In [5]:
# Optional: send the daily channel messages
# Set TEST_CHAT_ID in .env (example: your numeric chat_id or @TarotReaderOracle)

if TEST_CHAT_ID:
    bot = Bot(token=TELEGRAM_BOT_TOKEN)
    for message in channel_messages:
        await bot.send_message(TEST_CHAT_ID, message)
    await bot.session.close()
    "Sent"
else:
    "TEST_CHAT_ID not set"
